In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon
nltk.download('vader_lexicon')

# Load your dataset
df = pd.read_csv('cleaned_anime.csv')
# generes are set as lists in the cleaned dataset
df['genre'] = df['genre_tokens']

# Encode genre labels for easier conversion/analysis
# mlb allows us to look at multiple genres per listing
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

# Compute sentiment scores
sia = SentimentIntensityAnalyzer()
df['sentiment'] = df['clean_synopsis'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

# Combine genre features and sentiment
combined_features = pd.concat([genre_df, df['sentiment']], axis=1)

print("Combined Feature Vectors:")
print(combined_features.head())
# each row represents a movie w/ genre encoding (binary columns for each genre) and sentiment score (numeric value from -1 to +1)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Combined Feature Vectors:
      '  ,  [  ]  a  c  d  e  f  ...  n  o  p  r  s  t  u  v  y  sentiment
0  1  1  1  1  1  1  1  1  1  1  ...  1  1  0  1  1  1  1  1  0     0.8553
1  1  1  1  1  1  1  1  1  1  1  ...  1  1  0  1  1  1  0  0  1    -0.5635
2  1  1  1  1  1  1  1  1  1  1  ...  1  1  0  1  1  1  1  1  1    -0.9251
3  1  1  1  1  1  1  1  1  1  0  ...  1  1  1  1  1  1  1  0  1     0.6428
4  1  1  1  1  1  1  0  1  1  1  ...  1  0  1  1  1  1  1  1  1    -0.9578

[5 rows x 26 columns]


In [ ]:
# normalizing sentiment score to be a range of 0-1 to make calculation and display of cosine similarity matrix easier.
combined_features['sentiment'] = (combined_features['sentiment'] + 1) / 2

print("Combined Feature Vectors after sentiment normalization:")
print(combined_features.head())

Combined Feature Vectors after sentiment normalization:
      '  ,  [  ]  a  c  d  e  f  ...  n  o  p  r  s  t  u  v  y  sentiment
0  1  1  1  1  1  1  1  1  1  1  ...  1  1  0  1  1  1  1  1  0    0.92765
1  1  1  1  1  1  1  1  1  1  1  ...  1  1  0  1  1  1  0  0  1    0.21825
2  1  1  1  1  1  1  1  1  1  1  ...  1  1  0  1  1  1  1  1  1    0.03745
3  1  1  1  1  1  1  1  1  1  0  ...  1  1  1  1  1  1  1  0  1    0.82140
4  1  1  1  1  1  1  0  1  1  1  ...  1  0  1  1  1  1  1  1  1    0.02110

[5 rows x 26 columns]


In [ ]:
df['sentiment'] = combined_features['sentiment']
df['num_genres'] = genre_df.sum(axis=1)
# create new column for the genre score in addition to the sentiment score
print("Updated 'df' DataFrame with normalized sentiment and number of genres:")
print(df[['title', 'sentiment', 'num_genres']].head())

Updated 'df' DataFrame with normalized sentiment and number of genres:
                             title  sentiment  num_genres
0                     Cowboy Bebop    0.92765          19
1  Cowboy Bebop: Tengoku no Tobira    0.21825          18
2                           TRIGUN    0.03745          20
3               Witch Hunter ROBIN    0.82140          20
4                   Bouken Ou Beet    0.02110          18


In [ ]:
df['combined_score'] = df['num_genres'] + df['sentiment']
# make new final column that combines the scores of the genres and the sentiment
print("DataFrame with combined score:")
print(df[['title', 'num_genres', 'sentiment', 'combined_score']].head())

DataFrame with combined score:
                             title  num_genres  sentiment  combined_score
0                     Cowboy Bebop          19    0.92765        19.92765
1  Cowboy Bebop: Tengoku no Tobira          18    0.21825        18.21825
2                           TRIGUN          20    0.03745        20.03745
3               Witch Hunter ROBIN          20    0.82140        20.82140
4                   Bouken Ou Beet          18    0.02110        18.02110
